In this notebook we will continue to explore methods of learning a 1-dimensional function.


In [ ]:
import numpy as np
import scipy
import scipy.linalg
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interactive
import ipywidgets as widgets
from ipywidgets import fixed


Let's first define some utility methods that will be useful later. `generate_x` lets us produce a set of sample points over an interval, either evenly spaced, or drawn from a uniform distribution. The other functions are simple plotting utilities to be used later.


In [ ]:
def generate_x(n, x_type, x_low=-1, x_high=1):
    if x_type == 'grid':
        x = np.linspace(x_low, x_high, n, endpoint = False).astype(np.float64)

    elif x_type == 'uniform_random':
        x = np.sort(np.random.uniform(x_low, x_high, n).astype(np.float64))
        #Note that for making it easy for plotting we sort the randomly sampled x in ascending order
    else:
        raise ValueError


    return x


def plot_function(f):
    pts = generate_x(100, "grid")
    plt.plot(pts, f(pts))


def plot_features(featurize):
    samples = generate_x(100, 'grid')
    featurized_samples = featurize(samples)
    D = len(featurized_samples)
    for i, feature in enumerate(featurized_samples):
        plt.plot(samples, feature, label="Feature {}".format(i))
    plt.legend(loc="lower right")
    plt.show()


## Part (c)

In part (c), we will look at one way of orthonormalizing features under a given sample distribution. Let $p(x)$ define the probability density of the distribution of our sample points, and let $\phi_i(x)$ and $\phi_j(x)$ be two 1-dimensional feature functions, taking in a scalar input and returning a scalar output.

The inner product of these two features over our distribution is
$$
    \int \phi_i(x) \phi_j(x) p(x) \mathrm{d}x
$$
over the range of possible values of $x$.

We wish to choose features $\phi$ that have unit norm and are mutually orthogonal. One way to do this is to use the well-known Gram-Schmidt process.

First, let's look at the features of interest: the polynomial features $1$, $x$, and $x^2$.


In [ ]:
def polynomial_featurization(x, degree):
    n = x.shape[0]
    A = np.zeros((degree, n))
    for i in range(degree):
        A[i, :] = (x ** i).reshape((1, -1))
    return A

plot_features(lambda x: polynomial_featurization(x, 3))


In [ ]:
polynomial_featurization(np.array([1,2,3,4]), 3)


These functions will help us compute the covariance of our feature vector over a probability distribution.


In [ ]:
N = 10000

def compute_sample_covariance(features):
    return features.T @ features

def compute_feature_covariance(featurize, samples):
    # default distribution is uniform [-1, 1]
    if isinstance(samples, int):
        samples = generate_x(samples, "uniform_random")
    features = featurize(samples)
    cov = features @ np.conjugate(features.T) / len(samples) # divide by N to get the expected value of the covariance for N = 1
    return cov

covariance = compute_feature_covariance(lambda X: polynomial_featurization(X, 3), N)
plt.imshow(covariance)
covariance


You should notice that the covariance of the polynomial features is clearly not $I$, so they are not orthonormal. Use Gram-Schmidt to orthonormalize these features, and fill in the relevant lines in the below code block.


In [ ]:
def orthonormalized_quadratic_polynomial_featurization(x):
    n = x.shape[0]
    """
    The below block should look like
    ```
        polynomials = [
            lambda x: <FIRST POLYNOMIAL>,
            lambda x: <SECOND POLYNOMIAL>,
            lambda x: <THIRD POLYNOMIAL>,
        ]
    ```

    For instance, without any orthonormalization, it would simply be
    ```
        polynomials = [
            lambda x: 1,
            lambda x: x,
            lambda x: x**2,
        ]
    ```
    """
    polynomials = [
        ### start (c) ###

        ### end (c) ###
    ]
    A = np.zeros((3, n))
    for i, polynomial in enumerate(polynomials):
        A[i, :] = (polynomial(x) * np.ones(n)).reshape((1, -1))
    return A

plot_features(orthonormalized_quadratic_polynomial_featurization)


In [ ]:
covariance = compute_feature_covariance(lambda X: orthonormalized_quadratic_polynomial_featurization(X), N)
plt.imshow(covariance)
covariance


If you solved the problem correctly, you should see that the covariance matrix is essentially the $3 \times 3$ identity matrix (since we compute the covariance numerically, there may be some slight deviation due to randomness).


## Part (e)
We will now consider the case when the distribution of our test points $x$ is not uniform. The `weighted_distribution` function shows how we will draw points from the interval $[-1, 1]$. Our distribution will be piecewise uniform over $[-1, 0)$ and $(0, 1]$, but each sample point will be twice as likely to be positive, as to be negative.

Look at the covariance of our orthonormalized features from the previous part with respect to the uniform distribution, and with respect to this new `weighted_distribution`. Are they still orthogonal?


In [ ]:
def weighted_distribution(n):
    samples = generate_x(n, "uniform_random", -1, 2)
    samples[samples > 1] -= 1
    return samples

print("Covariance under uniform distribution:")
covariance = compute_feature_covariance(lambda X: orthonormalized_quadratic_polynomial_featurization(X), N)
print(covariance)
plt.imshow(covariance)
plt.show()

print("Covariance under p(x):")
covariance = compute_feature_covariance(lambda X: orthonormalized_quadratic_polynomial_featurization(X), weighted_distribution(N))
print(covariance)
plt.imshow(covariance)


We will now use an alternative method to orthonormalize a set of features with respect to an arbitrary `weighted_distribution`, as you derived in part (d) of the problem. We have implemented everything for you, all you have to do is run the below cell and understand the outputs.


In [ ]:
def orthonormalize(featurize, samples):
    K = compute_feature_covariance(featurize, samples)
    K_inv_square_root = np.linalg.inv(scipy.linalg.sqrtm(K))

    def featurization(samples):
        return K_inv_square_root @ featurize(samples)

    return featurization

N = 10000

orthonormalized_featurization = orthonormalize(
    lambda x: polynomial_featurization(x, 3),
    generate_x(N, "uniform_random", -1, 1),
)
print("Covariance under uniform distribution:")
covariance = compute_feature_covariance(orthonormalized_featurization, generate_x(N, "uniform_random", -1, 1))
print(covariance)
plt.imshow(covariance)
plt.show()

print("Orthonormalized quadratic features under uniform")
plot_features(orthonormalized_featurization)

orthonormalized_featurization = orthonormalize(
    lambda x: polynomial_featurization(x, 3),
    weighted_distribution(N),
)
print("Covariance under weighted distribution:")
covariance = compute_feature_covariance(orthonormalized_featurization, weighted_distribution(N))
print(covariance)
plt.imshow(covariance)
plt.show()

print("Orthonormalized quadratic features under weighted distribution")
plot_features(orthonormalized_featurization)


Consider the above orthonormalized features under the uniform distribution, and compare them to the orthonormalization you obtained earlier using Gram-Schmidt. Think about whether they are the same, and why that is or is not the case.


## Part (g)
From this point onwards, we will look at the Fourier features you studied in HW 1, rather than polynomial features. The `fourier_featurization` function featurizes scalar data points in a manner analogous to the `polynomial_featurization` function from earlier.


In [ ]:
def fourier_featurization(x, d):
    assert (d-1) % 2 == 0, "d must be odd"
    max_r = int((d-1)/2)
    n = len(x)
    A = np.zeros((d, n))
    A[0:, :] = 1
    for d_ in range(1,max_r+1):
        A[2*(d_-1)+1, :] =  np.sin(d_*x*np.pi)
        A[2*(d_-1)+2, :] =  np.cos(d_*x*np.pi)

    A[0, :] *= (1/np.sqrt(2))
    A *= np.sqrt(2)
    return A

plot_features(lambda x: fourier_featurization(x, 3))

fourier_featurization(np.array([1,2,3, 4]), 3)


As we showed in HW 1, properly normalized Fourier features are orthonormal with respect to a uniform distribution over $[-1, 1]$.


In [ ]:
covariance = compute_feature_covariance(lambda X: fourier_featurization(X, 3), 10000)
plt.imshow(covariance)
print(covariance)


The calculations in the last parts of this question assume that each of our features have zero-mean. All the Fourier features except for the first feature $1$ have this property, so we will remove it from our featurization.


In [ ]:
def zero_mean_fourier_featurization(x, d):
    assert d % 2 == 0
    return fourier_featurization(x, d + 1)[1:, :]

plot_features(lambda x: zero_mean_fourier_featurization(x, 2))


Our goal is to use these featurizations to learn a target function, using least squares on the featurized observations. For part (g), we will assume that our target function is simply $f(x) = \phi_1(x)$. `generate_weights` is a utility function that lets us generate random weights for use later on. `function_from_weights` converts a weight vector into a continuous function.

Observe our target function and the sample points we pick for our input data.


In [ ]:
def generate_weights(n):
    return np.random.uniform(-1, 1, n).astype(np.float64)

def function_from_weights(weights):
    n = len(weights)

    def f(x):
        features = zero_mean_fourier_featurization(x, n)
        return features.T @ weights
    return f

D = 10
num_samples = 10
true_weights = [1] + [0] * (D - 1)
true_function = function_from_weights(true_weights)

plot_function(true_function)
samples = generate_x(num_samples, "grid", -1, 1)
plt.scatter(samples, true_function(samples))


To make things interesting, we will introduce some noise into our sample points, so that when we perform least squares we will not exactly recover the true weights. Let's add this noise, and then perform least squares on the featurized data to try and recover our original weights. Since you've already done this in HW 1, we will do it for you here.


In [ ]:
def add_awgn_noise(y, awgn_std=0):
    noise = np.random.normal(0, awgn_std, y.shape)
    y_noisy = y + noise
    return y_noisy

noiseless_observations = true_function(samples)
noisy_observations = add_awgn_noise(noiseless_observations, 0.1)

featurized_samples = zero_mean_fourier_featurization(samples, D)

print("Noisy observations vs the true function")
plot_function(true_function)
plt.scatter(samples, noisy_observations)
plt.show()

# Now we will try to learn the function from the noisy observations
estimated_weights, *_ = np.linalg.lstsq(featurized_samples.T, noisy_observations, rcond=None)
estimated_function = function_from_weights(estimated_weights)

print("The estimated vs true function")
plt.scatter(samples, noisy_observations)
plot_function(true_function)
plot_function(estimated_function)


Hopefully, you should see an estimate that's reasonably close to the true function. We're not really concerned with the accuracy of our estimate, however, so don't worry much if it's not so accurate. (In HW1 you studied techniques like ridge regression which can help us improve our estimate).

To quantify the error of our estimate, we can compute the prediction error by computing the expected squared error in our prediction for a point drawn from the uniform distribution over $[-1, 1]$.


In [ ]:
def sample_error(samples, true_function, predicted_function):
    if isinstance(samples, int):
        samples = generate_x(samples, "uniform_random")
    return np.sum(np.abs(true_function(samples) - predicted_function(samples)) ** 2) / len(samples)

numerical_prediction_error = sample_error(1000, true_function, estimated_function)
print("Numerical prediction error:", numerical_prediction_error)


Fundamenally, this error is caused by two things. First, by the error in our estimate of the first weight (which, in the true function, was $1$). Second, by the error in our estimate of the other weights (which were really all just $0$). We will call the first source of error "survival" (`SU`) and the second source "contamination" (`CN`).

In this problem, you have shown that the error can be decomposed as
$$
    \mathcal{E}_{\text{pred}} = (1 - \text{SU})^2 + \text{CN}^2.
$$

Fill in the code blocks to compute the two sources of error, and verify that the above formula matches the numerical estimate of the prediction error obtained in the previous cell.


In [ ]:
def compute_SU(true_weights, predicted_weights):
    ### start (g)(1) ###

    ### end (g)(1) ###

def compute_CN(predicted_weights):
    ### start (g)(2) ###

    ### end (g)(2) ###

def expected_sample_error(true_weights, predicted_weights):
    return (1 - compute_SU(true_weights, predicted_weights))**2 + compute_CN(predicted_weights)**2

print("SU:", compute_SU(true_weights, estimated_weights))
print("CN:", compute_CN(estimated_weights))
print("Expected prediction error:", expected_sample_error(true_weights, estimated_weights))
print("Numerically estimated prediction error:", numerical_prediction_error)


## Part (h)

Finally, we will consider the case when our model is not powerful enough to learn the true function. Concretely, imagine that our true function is a linear combination of some $D$ features, but our prediction is a linear combination of a subset of size $d < D$ of these features. Then, it is clear that our prediction will not exactly match the true function, so there will be some error.

In this section, we will look at the various components of this error, and see how they depend on $d$ and $D$. We will divide the residual $R$ between our prediction and the true value into two components: the error caused by our incorrect estimation of the weights on the first $d$ features, and the error caused by our inability to estimate the weights on the last $D - d$ features, since they are not part of our model.

Concretely, in the case of zero-mean orthonormal features,
$$
    \mathcal{E}_{\text{pred}} = \sum_{j=1}^d (w_j^* - w_j)^2 + \text{Var}\left(\sum_{j=d+1}^D w_j^* \phi_j(X)\right) = \sum_{j=1}^d (w_j^* - w_j)^2 + \sum_{j=d+1}^D (w_j^*)^2.
$$

Run the below code block, and verify that our predicted value of $\mathcal{E}_{\text{pred}}$ matches what is calculated empirically as we vary $d$ and $\sigma$ where $\sigma^2$ is the variance of the AWGN in our training $y_i$.


In [ ]:
def compute_var_R1(true_weights, predicted_weights, featurized_samples):
    d = len(predicted_weights)
    return np.linalg.norm(true_weights[:d] - predicted_weights)**2

def compute_var_R2(true_weights, featurized_samples):
    d = featurized_samples.shape[0]
    return np.linalg.norm(true_weights[d:])**2

def estimate_true_function(d, sigma, true_weights):
    num_samples = 100

    D = len(true_weights)
    true_function = function_from_weights(true_weights)

    assert d < D

    # uniformly sample our true function with noise
    samples = generate_x(num_samples, "grid", -1, 1)
    noiseless_observations = true_function(samples)
    noisy_observations = add_awgn_noise(noiseless_observations, sigma)

    # do least squares using the features
    featurized_samples = zero_mean_fourier_featurization(samples, d)
    estimated_weights, *_ = np.linalg.lstsq(featurized_samples.T, noisy_observations, rcond=None)

    estimated_function = function_from_weights(estimated_weights)

    numerical_error = sample_error(1000, true_function, estimated_function)

    var_R1 = compute_var_R1(true_weights, estimated_weights, featurized_samples)
    var_R2 = compute_var_R2(true_weights, featurized_samples)
    expected_error = var_R1 + var_R2

    return var_R1, var_R2, numerical_error, expected_error

def plot_estimates(D, sigma):
    # generate a true function
    true_weights = generate_weights(D)

    ds = range(2, D, 2)
    var_R1s = []
    var_R2s = []
    numerical_errors = []
    expected_errors = []
    for d in ds:
        var_R1, var_R2, numerical_error, expected_error = estimate_true_function(d, sigma, true_weights)
        var_R1s.append(var_R1)
        var_R2s.append(var_R2)
        numerical_errors.append(numerical_error)
        expected_errors.append(expected_error)

    plt.plot(ds, var_R1s, label="Var(R_1)")
    plt.plot(ds, var_R2s, label="Var(R_2)")
    plt.plot(ds, expected_errors, label="E_pred")
    plt.xlabel("$d$")
    plt.legend()
    plt.show()

    plt.plot(ds, numerical_errors, label="E_numerical")
    plt.plot(ds, expected_errors, label="E_pred")
    plt.xlabel("$d$")
    plt.legend()
    plt.show()

slider = widgets.FloatSlider(
    value=3,
    min=0,
    max=10,
    step=0.1,
    description='$\sigma$',
    continuous_update=False
)

D = 100
print("D = ", D)
interactive_plot = interactive(plot_estimates, D=fixed(D), sigma=slider)
interactive_plot
